In [3]:
import pandas as pd
import numpy as np
from astropy.time import Time

In [4]:
from alerce.api import AlerceAPI
client = AlerceAPI()

In [5]:
tns=pd.read_csv('tns_d.csv') #Fron 10/12/19 to 31/5/20

In [6]:
#/////////MATCH X COORD
def tnsxmatch (snlist):
    tnsxm=pd.DataFrame()
    tol=5*0.00027777777777778
    for oid, ob in snlist.iterrows():
        xde=tns[(np.isclose(tns.dec, ob.meandec, atol=tol,rtol=0.0))]
        mx=xde[np.isclose(xde.ra,ob.meanra,atol=tol,rtol=0.0)]
        if mx.shape[0]!=0:
            if mx.shape[0]>1:   #select nearest if there's more than 1 match
                m=np.full(mx.shape[0],False)
                m[((mx.dec- ob.meandec)**2+(mx.ra- ob.meanra)**2).argmin()]=True
                mx=mx[m]
            mx['oid']=oid
            if len(tnsxm)>0 and mx.index[0] in tnsxm.index:
                if mx['Disc. Internal Name'].values[0]==oid:
                    tnsxm['oid']=tnsxm['oid'].replace(tnsxm[tnsxm['Disc. Internal Name']==oid]['oid'],oid)
                continue
            tnsxm=tnsxm.append(mx)
     
    print(len(tnsxm))
    return(tnsxm)

In [7]:
#/////////// ALeRCE objecst classified as SN by stamp classifier
records_per_page = 100000
min_nobs = 10
max_nobs = 50
min_firstmjd = 58828 
max_firstmjd=59000.863 #may 31
classearly=19 
params = {
    "records_per_pages": records_per_page,
    "page": 1,
    #"sortBy": sortBy,
    "query_parameters": {
        "filters": {
            #"classrf": 10,
            "classearly":classearly,
            "nobs": {
                #"min": min_nobs,
                #"max": max_nobs
            }
        },
            "dates": {
                "firstmjd": {
                    "min": min_firstmjd,
                    "max": max_firstmjd
                }
            }
    }
}

In [8]:
SNe = client.query(params, format='pandas')
print(SNe.shape)

(79542, 43)


In [9]:
tnsxm=tnsxmatch(SNe)

/home/emilio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


3830


In [10]:
repbyalerce=sum(['ALeRCE' in s for s in tnsxm['Reporting Group/s'].values])
repbyZTF=sum(['ZTF' in s for s in tnsxm['Discovery Data Source/s'].values])

print(len(SNe),'objects classified as SN by stamp classifier')
print(len(tnsxm), 'reported to TNS')
print(repbyalerce,'reported to TNS by ALeRCE,',repbyalerce*100/len(tnsxm),'%')
print(repbyZTF-repbyalerce,'reported by other group from ZTF data')

79542 objects classified as SN by stamp classifier
3830 reported to TNS
1733 reported to TNS by ALeRCE, 45.24804177545692 %
1657 reported by other group from ZTF data


In [11]:
#objects reported with ZTF data
tns_ztf=tns[[(('ZTF' in s)if str(s) != 'nan' else False )for s in tns['Discovery Data Source/s'].values]]
ztfrepbyalerce=sum(['ALeRCE' in s for s in tns_ztf['Reporting Group/s'].values])
print(len(tns),'objects total reported to tns in the span of time')
print(len(tns_ztf), 'fron ZTF data')
print(ztfrepbyalerce,'reported by ALeRCE')

10031 objects total reported to tns in the span of time
4726 fron ZTF data
1752 reported by ALeRCE
